# Prepare Port Scanning Closed Port Attack Dataset

## Overview:

This notebook will focus on creating a Port Scanning closed port attack dataset based on a small sample of data collected by performing real Port Scanning closed port attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
There are multiple sample datasets because we performed the attack in a few different ways, and in each way, the data is slightly different.<br>
That is why we split the original sample dataset into multiple samples, ensuring that the attack dataset we generate matches the real-world data as closely as possible.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 20,000 flows of the Port Scanning closed port attack based on the samples we collected when running a Port Scanning attack in various configurations using the well known NMap tool when the majority of ports on the victim host machine where closed.<br> 

## Imports & Global Variables:

In [2]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 15000
ATTACK_NAME = 'PortScan'

In [3]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the first sample dataset:

In [4]:
# import the attack sample dataset
port_samples = pd.read_csv('portscan_closed_port_samples_1.csv')
print(f'Dataset Shape: {port_samples.shape}')
port_samples

Dataset Shape: (19, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1371,60.0,60,60,0.0,0.0,71266,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,71266.0,2741,0,0,37.678743,72.746588,1.111484,0.013751,0.047234
1,1141,60.0,60,60,0.0,0.0,59306,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,59306.0,2281,0,0,38.145140,59.797919,1.215345,0.016730,0.062343
2,1545,74.0,74,74,0.0,0.0,123000,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,0.0,3075,0,0,39.945872,76.979169,0.385716,0.012995,0.040512
3,1347,74.0,74,74,0.0,0.0,111320,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,55660.0,2783,0,0,39.997725,69.578957,1.706628,0.014377,0.057339
4,1161,74.0,74,74,0.0,0.0,92480,40,40.0,40,0.0,0,0.0,0,0.0,0.0,0.0,92480.0,2312,0,0,38.068128,60.733220,1.218413,0.016473,0.058114
5,1170,60.0,60,60,0.0,0.0,60632,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,60632.0,2332,0,0,36.205047,64.410909,1.153600,0.015532,0.055096
6,1330,60.0,60,60,0.0,0.0,68926,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,68926.0,2651,0,0,37.214555,71.235569,1.151637,0.014043,0.048594
7,1710,60.0,60,60,0.0,0.0,88764,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,88764.0,3414,0,0,35.508218,96.146757,1.096689,0.010404,0.031219
8,1700,60.0,60,60,0.0,0.0,88348,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,88348.0,3398,0,0,35.340668,96.149852,1.102454,0.010403,0.028880
9,1750,60.0,60,60,0.0,0.0,90844,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,90844.0,3494,0,0,36.337302,96.154635,1.101902,0.010403,0.030542


### Find the columns that we need to synthesis data for:

In [5]:
columns_to_gather = port_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [6]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 15%), and store the results in a dictionary.
min_max_dict = {col: (port_samples[col].min() * 0.85, port_samples[col].max() * 1.15) for col in columns_to_gather}

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 969.85 | Max: 2285.05
Average Packet Length          | Min: 51.00 | Max: 85.10
Packet Length Min              | Min: 51.00 | Max: 85.10
Packet Length Max              | Min: 51.00 | Max: 85.10
Total Length of Fwd Packet     | Min: 50410.10 | Max: 175260.00
Fwd Packet Length Max          | Min: 22.10 | Max: 46.00
Fwd Packet Length Mean         | Min: 22.10 | Max: 46.00
Fwd Packet Length Min          | Min: 22.10 | Max: 46.00
Fwd Segment Size Avg           | Min: 0.00 | Max: 2.30
Subflow Fwd Bytes              | Min: 0.00 | Max: 175260.00
SYN Flag Count                 | Min: 1938.85 | Max: 4550.55
Flow Duration                  | Min: 30.04 | Max: 46.00
Packets Per Second             | Min: 50.83 | Max: 113.98
IAT Max                        | Min: 0.05 | Max: 1.96
IAT Mean                       | Min: 0.01 | Max: 0.02
IAT Std                        | Min: 0.01 | Max: 0.07


### Create the base attack dataset (full of zeros):

In [7]:
# creating an empty dataframe before adding values to it
port_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(port_samples.columns))), columns = port_samples.columns)
port_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [8]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in port_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    port_dataset[col] = int(0)
zero_columns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

In [9]:
port_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


---

## Filling in values based on collected samples:

## Calculate and fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **flow duration increases**, the **packets per second** is likely to decrease. This occurs because each flow has an upper limit on duration, after which data collection stops and a new flow begins.<br>  
Similarly, the **Inter-Arrival Time (IAT)** of packets within a flow is influenced by the flow duration. Given these dependencies, <br>
the attack dataset should generate data for these columns collectively, ensuring that their inherent correlations are maintained.

### Correlation between 'Number of Ports' and all the following: 'Total Length of Fwd Packet', 'SYN Flag Count':

In [10]:
# finding the correlation between the 'Number of Ports' column to the rest of the columns in order to create new data
first_correlation = ['Number of Ports', 'Total Length of Fwd Packet', 'SYN Flag Count']
independent_col = port_samples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = port_samples[first_correlation[1:]].values  

# using least squares regression to find scaling factors that best approximate the relationship between 'Number of Ports' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Total Length of Fwd Packet', np.float64(62.472578636706345))
('SYN Flag Count', np.float64(2.0004714171156874))


After finding the scaling factors we can apply some randomness when generating values for the attack dataset in order to generate better data (without many duplications).<br>
We add randomness by creating a modified scaling factor, which introduces controlled variations in the generated values.<br>
This is done by selecting a small random delta (between 1% and 2% of the factor) and adding or subtracting it from the original scaling factor.<br>
As a result, the generated data maintains realistic correlations while avoiding exact duplicates.

In [11]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
port_dataset['Number of Ports'] = np.random.randint(min_max_dict['Number of Ports'][0]*0.9, min_max_dict['Number of Ports'][1]*1.10, NUM_OF_ROWS)

# generate new data by scaling the original correlated column value using the updated factor.
for index, row in port_dataset.iterrows():
    for col, factor in zip(first_correlation[1:], scaling_factors): #iterating over all generated scaling factors
        delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02)
        updated_factor = factor[1] + random.choice([-1, 1]) * delta
        port_dataset.loc[index, col] = row['Number of Ports'] * updated_factor

In [12]:
port_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1281,0.0,0.0,0.0,0,0,81275.857844,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2593.771643,0,0,0.0,0.0,0.0,0.0,0.0
1,2401,0.0,0.0,0.0,0,0,152571.219198,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4743.957800,0,0,0.0,0.0,0.0,0.0,0.0
2,2371,0.0,0.0,0.0,0,0,145918.773245,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4691.425224,0,0,0.0,0.0,0.0,0.0,0.0
3,1534,0.0,0.0,0.0,0,0,97431.036766,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,3009.920035,0,0,0.0,0.0,0.0,0.0,0.0
4,2124,0.0,0.0,0.0,0,0,131333.271926,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4204.481648,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2301,0.0,0.0,0.0,0,0,145266.487566,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4681.130760,0,0,0.0,0.0,0.0,0.0,0.0
14996,1519,0.0,0.0,0.0,0,0,95924.409172,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,3097.950577,0,0,0.0,0.0,0.0,0.0,0.0
14997,1247,0.0,0.0,0.0,0,0,76643.707453,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2526.233846,0,0,0.0,0.0,0.0,0.0,0.0
14998,1475,0.0,0.0,0.0,0,0,93799.113677,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2994.650149,0,0,0.0,0.0,0.0,0.0,0.0


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [13]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = port_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = port_samples[second_correlation[1:]].values  

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(2.249872988044725))
('IAT Max', np.float64(0.02663488227646748))
('IAT Mean', np.float64(0.0003135198870679136))
('IAT Std', np.float64(0.0010030869957642617))


In [14]:
# generate random values for the 'Flow Duration' column
rand_values = np.random.uniform(min_max_dict['Flow Duration'][0]*0.9, min_max_dict['Flow Duration'][1]*1.05, size = NUM_OF_ROWS)

# assign the random values
port_dataset['Flow Duration'] = rand_values

port_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1281,0.0,0.0,0.0,0,0,81275.857844,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2593.771643,0,0,27.748915,0.0,0.0,0.0,0.0
1,2401,0.0,0.0,0.0,0,0,152571.219198,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4743.957800,0,0,34.565506,0.0,0.0,0.0,0.0
2,2371,0.0,0.0,0.0,0,0,145918.773245,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4691.425224,0,0,37.063791,0.0,0.0,0.0,0.0
3,1534,0.0,0.0,0.0,0,0,97431.036766,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,3009.920035,0,0,41.697516,0.0,0.0,0.0,0.0
4,2124,0.0,0.0,0.0,0,0,131333.271926,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4204.481648,0,0,38.720914,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2301,0.0,0.0,0.0,0,0,145266.487566,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,4681.130760,0,0,29.186483,0.0,0.0,0.0,0.0
14996,1519,0.0,0.0,0.0,0,0,95924.409172,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,3097.950577,0,0,44.399071,0.0,0.0,0.0,0.0
14997,1247,0.0,0.0,0.0,0,0,76643.707453,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2526.233846,0,0,29.288432,0.0,0.0,0.0,0.0
14998,1475,0.0,0.0,0.0,0,0,93799.113677,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,2994.650149,0,0,39.357523,0.0,0.0,0.0,0.0


In [15]:
# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(port_samples['Flow Duration'].values, port_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

np.float64(3290.4736842105262)

And again here after finding the scaling factors we add some randomness and generate the data

In [16]:
# calculate a random small delta of the factor for some randomness
for index, row in port_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr * 0.25, duration_to_packets_corr * 0.65) 
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            port_dataset.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.35, factor * 0.65)
                updated_factor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) 
                updated_factor = factor + random.choice([-1, 1]) * delta

            if col == 'IAT Max':
                delta = random.uniform(factor * 0.3, factor * 0.6)
                updated_factor = factor + random.choices([-1, 1], weights=[1, 2], k=1)[0] * delta  
                port_dataset.loc[index, col] = (row['Flow Duration'] * updated_factor)
            else:
                port_dataset.loc[index, col] = row['Flow Duration'] * updated_factor

### Then we insert data into the 'Fwd Segment Size Avg' column:

This column also either has a zero or a number, and we can see in the sample data that the zeros coinside with the zeros in the 'Packet Length Std' and 'Packet Length Variance' columns, there for we use the same mask to insert the zeros into the same cell indexes as the other columns.

In [ ]:
mask = np.random.rand(NUM_OF_ROWS) > 0.5 #randomly choose 50% of the cells in the vector
fwd_segment_size = port_samples[port_samples['Fwd Segment Size Avg'] != 0]['Fwd Segment Size Avg'] 
min_max_dict['Fwd Segment Size Avg'] = (np.min(fwd_segment_size), np.max(fwd_segment_size))

port_dataset['Fwd Segment Size Avg'] = np.random.uniform(min_max_dict['Fwd Segment Size Avg'][0]*0.9, min_max_dict['Fwd Segment Size Avg'][1]*1.1, size = NUM_OF_ROWS)
port_dataset.loc[mask, 'Fwd Segment Size Avg'] = 0

In [18]:
port_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1281,0.0,0.0,0.0,0,0,81275.857844,0.0,0.0,0.0,0,0,0,0,0,0.000000,0,0.0,2593.771643,0,0,27.748915,87.515852,1.033095,0.009642,0.045105
1,2401,0.0,0.0,0.0,0,0,152571.219198,0.0,0.0,0.0,0,0,0,0,0,1.903492,0,0.0,4743.957800,0,0,34.565506,58.392218,1.251778,0.012928,0.012403
2,2371,0.0,0.0,0.0,0,0,145918.773245,0.0,0.0,0.0,0,0,0,0,0,2.005345,0,0.0,4691.425224,0,0,37.063791,129.055878,0.413607,0.010060,0.059894
3,1534,0.0,0.0,0.0,0,0,97431.036766,0.0,0.0,0.0,0,0,0,0,0,2.017997,0,0.0,3009.920035,0,0,41.697516,31.578013,1.694911,0.014539,0.019767
4,2124,0.0,0.0,0.0,0,0,131333.271926,0.0,0.0,0.0,0,0,0,0,0,2.189667,0,0.0,4204.481648,0,0,38.720914,126.194020,1.491597,0.010745,0.024609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2301,0.0,0.0,0.0,0,0,145266.487566,0.0,0.0,0.0,0,0,0,0,0,0.000000,0,0.0,4681.130760,0,0,29.186483,65.898281,1.125483,0.008120,0.042492
14996,1519,0.0,0.0,0.0,0,0,95924.409172,0.0,0.0,0.0,0,0,0,0,0,0.000000,0,0.0,3097.950577,0,0,44.399071,119.365207,1.634083,0.012183,0.068397
14997,1247,0.0,0.0,0.0,0,0,76643.707453,0.0,0.0,0.0,0,0,0,0,0,1.820927,0,0.0,2526.233846,0,0,29.288432,177.087630,1.178271,0.010840,0.013439
14998,1475,0.0,0.0,0.0,0,0,93799.113677,0.0,0.0,0.0,0,0,0,0,0,0.000000,0,0.0,2994.650149,0,0,39.357523,129.096275,0.552642,0.011078,0.063900


### Then we insert data into columns that have the exact same values:

The following columns have values that are exactly the same, there for we generate a single vector and insert into into all of these columns.

In [ ]:
same_values = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']

# generate random values for the 'Flow Duration' column
rand_values = np.random.randint(min_max_dict['Fwd Packet Length Max'][0]*0.9, min_max_dict['Fwd Packet Length Max'][1]*1.1, size = NUM_OF_ROWS)

# assign the random values
for col in same_values:
    port_dataset[col] = rand_values

In [ ]:
same_values2 = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']

# generate random values for the 'Flow Duration' column
rand_values = np.random.randint(min_max_dict['Average Packet Length'][0]*0.9, min_max_dict['Average Packet Length'][1]*1.1, size = NUM_OF_ROWS)

# assign the random values
for col in same_values2:
    port_dataset[col] = rand_values

In our sample dataset, the column 'Subflow Fwd Bytes' usually has values in a specific range, but sometimes it has zero values.<br>
In order to generate accurate data, we generate a vector that will have a certain distribution of values. For example, in the 'Subflow Fwd Bytes' column, <br>
50% of the values will be within the usual range, but the other 50% will have zero values.  

In [21]:
# generate a vector with random values based on min max dict, and also create a zero vector
col = 'Subflow Fwd Bytes'
subflow_values = port_samples[port_samples[col] != 0][col] 
min_max_dict[col] = (np.min(subflow_values), np.max(subflow_values))

rand_values = np.random.uniform(min_max_dict[col][0]*0.9, min_max_dict[col][1]*1.1, NUM_OF_ROWS)
zero_values = np.zeros(NUM_OF_ROWS)

# choose values randomly (50% from rand_values, 50% from zero_values)
port_dataset[col] = np.where(np.random.rand(NUM_OF_ROWS) > 0.5, rand_values, zero_values)

---

## Adding the Label column and adjusting some columns to have integer values:

In [22]:
# making the SYN Flag Count column have int values instead of floats
port_dataset['SYN Flag Count'] = port_dataset['SYN Flag Count'].astype(int)

# adding a label to the dataset
port_dataset['Label'] = ATTACK_NAME

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [23]:
port_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1281,67,67,67,0,0,81275.857844,42,42,42,0,0,0,0,0,0.000000,0,98811.811224,2593,0,0,27.748915,87.515852,1.033095,0.009642,0.045105,PortScan
1,2401,69,69,69,0,0,152571.219198,25,25,25,0,0,0,0,0,1.903492,0,71693.015059,4743,0,0,34.565506,58.392218,1.251778,0.012928,0.012403,PortScan
2,2371,89,89,89,0,0,145918.773245,23,23,23,0,0,0,0,0,2.005345,0,62066.325263,4691,0,0,37.063791,129.055878,0.413607,0.010060,0.059894,PortScan
3,1534,54,54,54,0,0,97431.036766,26,26,26,0,0,0,0,0,2.017997,0,0.000000,3009,0,0,41.697516,31.578013,1.694911,0.014539,0.019767,PortScan
4,2124,91,91,91,0,0,131333.271926,22,22,22,0,0,0,0,0,2.189667,0,0.000000,4204,0,0,38.720914,126.194020,1.491597,0.010745,0.024609,PortScan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2301,55,55,55,0,0,145266.487566,45,45,45,0,0,0,0,0,0.000000,0,0.000000,4681,0,0,29.186483,65.898281,1.125483,0.008120,0.042492,PortScan
14996,1519,48,48,48,0,0,95924.409172,36,36,36,0,0,0,0,0,0.000000,0,91172.641761,3097,0,0,44.399071,119.365207,1.634083,0.012183,0.068397,PortScan
14997,1247,48,48,48,0,0,76643.707453,35,35,35,0,0,0,0,0,1.820927,0,0.000000,2526,0,0,29.288432,177.087630,1.178271,0.010840,0.013439,PortScan
14998,1475,92,92,92,0,0,93799.113677,39,39,39,0,0,0,0,0,0.000000,0,130252.055729,2994,0,0,39.357523,129.096275,0.552642,0.011078,0.063900,PortScan


In [24]:
port_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,19.000000,19.000000,19.000000,19.000000,19.0,19.0,19.000000,19.000000,19.000000,19.000000,19.0,19.0,19.0,19.0,19.0,19.000000,19.0,19.000000,19.000000,19.0,19.0,19.000000,19.000000,19.000000,19.000000,19.000000
mean,1644.526316,65.157895,65.157895,65.157895,0.0,0.0,102672.842105,31.157895,31.157895,31.157895,0.0,0.0,0.0,0.0,0.0,1.263158,0.0,79974.105263,3290.473684,0.0,0.0,38.213810,86.061046,1.023122,0.012007,0.038436
std,295.965533,6.938325,6.938325,6.938325,0.0,0.0,30054.279306,6.938325,6.938325,6.938325,0.0,0.0,0.0,0.0,0.0,0.991189,0.0,48727.358127,588.773336,0.0,0.0,1.530809,14.758534,0.411222,0.002376,0.013518
min,1141.000000,60.000000,60.000000,60.000000,0.0,0.0,59306.000000,26.000000,26.000000,26.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2281.000000,0.0,0.0,35.340668,59.797919,0.056668,0.010092,0.011413
25%,1359.000000,60.000000,60.000000,60.000000,0.0,0.0,88556.000000,26.000000,26.000000,26.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,57483.000000,2762.000000,0.0,0.0,37.393379,71.991078,1.097926,0.010344,0.030881
50%,1750.000000,60.000000,60.000000,60.000000,0.0,0.0,96512.000000,26.000000,26.000000,26.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,88348.000000,3494.000000,0.0,0.0,38.145140,96.149852,1.102612,0.010403,0.034800
75%,1890.000000,74.000000,74.000000,74.000000,0.0,0.0,117160.000000,40.000000,40.000000,40.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,96213.000000,3775.000000,0.0,0.0,39.337412,96.698718,1.152619,0.013897,0.047914
max,1987.000000,74.000000,74.000000,74.000000,0.0,0.0,152400.000000,40.000000,40.000000,40.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,152400.000000,3957.000000,0.0,0.0,39.997725,99.110343,1.706628,0.016730,0.062343


In [25]:
port_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.0,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.0,15000.0,15000.0,15000.0,15000.0,15000.000000,15000.0,15000.000000,15000.000000,15000.0,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,1686.903533,68.664667,68.664667,68.664667,0.0,0.0,105378.691057,34.097267,34.097267,34.097267,0.0,0.0,0.0,0.0,0.0,1.014815,0.0,52709.025353,3374.775000,0.0,0.0,37.722518,89.919057,1.151715,0.011805,0.037682
std,471.821920,13.798255,13.798255,13.798255,0.0,0.0,29509.604519,8.921669,8.921669,8.921669,0.0,0.0,0.0,0.0,0.0,1.003560,0.0,58526.916274,945.585465,0.0,0.0,6.150022,44.910634,0.482087,0.002647,0.020259
min,872.000000,45.000000,45.000000,45.000000,0.0,0.0,53493.987708,19.000000,19.000000,19.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1718.000000,0.0,0.0,27.036350,23.883697,0.293734,0.006815,0.009609
25%,1275.750000,57.000000,57.000000,57.000000,0.0,0.0,79686.309617,26.000000,26.000000,26.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2557.000000,0.0,0.0,32.323813,48.013080,0.633632,0.009890,0.018504
50%,1688.000000,69.000000,69.000000,69.000000,0.0,0.0,105386.104554,34.000000,34.000000,34.000000,0.0,0.0,0.0,0.0,0.0,1.805870,0.0,0.000000,3381.000000,0.0,0.0,37.771410,88.755416,1.240976,0.011512,0.030172
75%,2092.000000,80.000000,80.000000,80.000000,0.0,0.0,130463.336814,42.000000,42.000000,42.000000,0.0,0.0,0.0,0.0,0.0,2.004283,0.0,105411.577047,4180.000000,0.0,0.0,43.067087,126.880219,1.560743,0.013585,0.056357
max,2512.000000,92.000000,92.000000,92.000000,0.0,0.0,159852.594323,49.000000,49.000000,49.000000,0.0,0.0,0.0,0.0,0.0,2.200000,0.0,167600.055720,5124.000000,0.0,0.0,48.294568,199.859331,2.046560,0.018126,0.079478


---

## Load the second sample dataset:

The following code will create another attack dataset, this time based on a different sample dataset, the code in this section<br> 
will be mostly the same as it was up until this point in the notebook, there for we will not repeat the same explanations here.<br>  
For the second sample we intentionally generate more rows then we need because at the end we will be selecting 5,000 rows that fit our needs out of this second dataset. The rows we will take will be rows that have 'Number of Ports' >= 170.

In [26]:
NUM_OF_ROWS = 15000

## Load the second sample dataset:

In [27]:
# import the attack sample dataset
port_samples = pd.read_csv('portscan_closed_port_samples_2.csv')
print(f'Dataset Shape: {port_samples.shape}')
port_samples

Dataset Shape: (10, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,250,60.0,60,60,0.0,0.0,13000,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,13000.000000,500,0,0,5.972138,83.722109,1.099297,0.011968,0.057015
1,200,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,10400.000000,400,0,0,4.944851,80.892222,1.101934,0.012393,0.061607
2,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.135300,70.168720,1.104636,0.014316,0.078960
3,100,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,3466.666667,400,0,0,10.887169,36.740497,5.026671,0.027286,0.263723
4,150,60.0,60,60,0.0,0.0,7800,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,7800.000000,300,0,0,3.943459,76.075343,1.102673,0.013189,0.069544
5,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.124340,70.414876,1.100336,0.014266,0.078805
6,130,60.0,60,60,0.0,0.0,6760,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,6760.000000,260,0,0,3.534727,73.555891,1.103546,0.013648,0.073640
7,115,60.0,60,60,0.0,0.0,5980,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5980.000000,230,0,0,3.329684,69.075624,1.103299,0.014540,0.077855
8,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.141675,70.026340,1.100339,0.014346,0.078919
9,160,60.0,60,60,0.0,0.0,8320,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,8320.000000,320,0,0,4.147076,77.162799,1.101572,0.013000,0.066671


### Find the columns that we need to synthesis data for:

In [28]:
# find the columns that we need to synthesis data for to produce an attack dataset
columns_to_gather = port_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [29]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 15%), and store the results in a dictionary.
min_max_dict = {col: (port_samples[col].min(), port_samples[col].max() * 1.5) for col in columns_to_gather}

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 100.00 | Max: 375.00
Average Packet Length          | Min: 60.00 | Max: 90.00
Packet Length Min              | Min: 60.00 | Max: 90.00
Packet Length Max              | Min: 60.00 | Max: 90.00
Total Length of Fwd Packet     | Min: 5720.00 | Max: 19500.00
Fwd Packet Length Max          | Min: 26.00 | Max: 39.00
Fwd Packet Length Mean         | Min: 26.00 | Max: 39.00
Fwd Packet Length Min          | Min: 26.00 | Max: 39.00
Fwd Segment Size Avg           | Min: 2.00 | Max: 3.00
Subflow Fwd Bytes              | Min: 3466.67 | Max: 19500.00
SYN Flag Count                 | Min: 220.00 | Max: 750.00
Flow Duration                  | Min: 3.12 | Max: 16.33
Packets Per Second             | Min: 36.74 | Max: 125.58
IAT Max                        | Min: 1.10 | Max: 7.54
IAT Mean                       | Min: 0.01 | Max: 0.04
IAT Std                        | Min: 0.06 | Max: 0.40


### Create the base attack dataset (full of zeros):

In [30]:
# creating an empty dataframe before adding values to it
port_dataset2 = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(port_samples.columns))), columns = port_samples.columns)

### Find the columns with constant zero values based on samples:

In [31]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in port_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    port_dataset2[col] = int(0)
zero_columns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

---

## Filling in values based on collected samples:

## Calculate and fill values into columns that have a certain correlation between them:

### Correlation between 'Number of Ports' and all the following: 'Total Length of Fwd Packet', 'SYN Flag Count':

In [32]:
first_correlation = ['Number of Ports', 'Total Length of Fwd Packet', 'SYN Flag Count']

# finding the correlation between the 'Number of Ports' column to the rest of the columns in order to create new data
independent_col = port_samples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = port_samples[first_correlation[1:]].values  

# using least squares regression to find scaling factors that best approximate the relationship between 'Number of Ports' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
port_dataset2['Number of Ports'] = np.random.randint(min_max_dict['Number of Ports'][0]*0.9, min_max_dict['Number of Ports'][1]*1.10, NUM_OF_ROWS)

# generate new data by scaling the original correlated column value using the updated factor.
for index, row in port_dataset2.iterrows():
    for col, factor in zip(first_correlation[1:], scaling_factors): #iterating over all generated scaling factors
        delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02) 
        updated_factor = factor[1] + random.choice([-1, 1]) * delta
        port_dataset2.loc[index, col] = int(row['Number of Ports'] * updated_factor)

('Total Length of Fwd Packet', np.float64(54.290496641339054))
('SYN Flag Count', np.float64(2.088096024666887))


### Correlation between 'Number of Ports' and all of the following: 'Flow Duration', 'IAT Mean', 'IAT Std':

In [33]:
second_correlation = ['Number of Ports', 'Flow Duration', 'IAT Mean', 'IAT Std', 'Subflow Fwd Bytes'] 

# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
independent_col = port_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = port_samples[second_correlation[1:]].values  

# using least squares regression to find scaling factors that best approximate the relationship between 'Number of Ports' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Flow Duration', np.float64(0.029522499855939625))
('IAT Mean', np.float64(8.9972788366408e-05))
('IAT Std', np.float64(0.000522445844828285))
('Subflow Fwd Bytes', np.float64(51.236501119553644))


In [ ]:
# iterating over all rows we need to add values
for index, row in port_dataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all generated scaling factors
        if col == 'Flow Duration':
            delta = random.uniform(factor * 0.05, factor * 0.1) 
        elif col == 'IAT Std':
            delta = random.uniform(factor * 0.05, factor * 0.2) * random.choice([-1, 1]) 
        else:
            delta = random.uniform(factor * 0.1, factor * 0.25) 
        updated_factor = factor + delta
        port_dataset2.loc[index, col] = row['Number of Ports'] * updated_factor

### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max':

In [35]:
port_samples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,250,60.0,60,60,0.0,0.0,13000,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,13000.000000,500,0,0,5.972138,83.722109,1.099297,0.011968,0.057015
1,200,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,10400.000000,400,0,0,4.944851,80.892222,1.101934,0.012393,0.061607
2,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.135300,70.168720,1.104636,0.014316,0.078960
3,100,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,3466.666667,400,0,0,10.887169,36.740497,5.026671,0.027286,0.263723
4,150,60.0,60,60,0.0,0.0,7800,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,7800.000000,300,0,0,3.943459,76.075343,1.102673,0.013189,0.069544
5,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.124340,70.414876,1.100336,0.014266,0.078805
6,130,60.0,60,60,0.0,0.0,6760,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,6760.000000,260,0,0,3.534727,73.555891,1.103546,0.013648,0.073640
7,115,60.0,60,60,0.0,0.0,5980,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5980.000000,230,0,0,3.329684,69.075624,1.103299,0.014540,0.077855
8,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.141675,70.026340,1.100339,0.014346,0.078919
9,160,60.0,60,60,0.0,0.0,8320,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,8320.000000,320,0,0,4.147076,77.162799,1.101572,0.013000,0.066671


In [36]:
# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(port_samples['Flow Duration'].values, port_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

np.float64(307.0)

In [37]:
for index, row in port_dataset.iterrows():
    delta = random.uniform(duration_to_packets_corr * 0.25, duration_to_packets_corr * 0.65) 
    updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
    port_dataset2.loc[index, 'Packets Per Second'] = updated_factor / row['Flow Duration']

In [38]:
packets_per_second = 62.5 + (port_dataset2['Flow Duration'] - 2.0) * (35 / 7.5) #linear transformation
port_dataset2['Packets Per Second'] = np.clip(packets_per_second, 62.5, 88.75) #ensure within range

iat_max = 1.100 + (port_dataset2['Flow Duration'] - 2.0) * (0.013 / 7.5) + np.random.uniform(-0.002, 0.002, size = NUM_OF_ROWS)
port_dataset2['IAT Max'] = np.clip(iat_max, 1.100, 1.113) #ensure within range

In [39]:
x = port_dataset2[port_dataset2['Number of Ports'] > 145]
x[x['Number of Ports'] < 155].head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
5,147,0.0,0.0,0.0,0,0,8103.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,9091.254607,303.0,0,0,4.592477,74.598225,1.106090,0.016388,0.088956
41,152,0.0,0.0,0.0,0,0,8400.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,9674.419771,321.0,0,0,4.869362,75.890357,1.103037,0.016317,0.092314
58,151,0.0,0.0,0.0,0,0,8063.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,8526.629801,309.0,0,0,4.863582,75.863382,1.104102,0.015477,0.090058
84,150,0.0,0.0,0.0,0,0,8014.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,8675.696045,308.0,0,0,4.822555,75.671924,1.104927,0.014958,0.093285
115,150,0.0,0.0,0.0,0,0,8229.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,9526.032316,307.0,0,0,4.795388,75.545143,1.106442,0.015170,0.062835
138,147,0.0,0.0,0.0,0,0,7862.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,8777.186851,301.0,0,0,4.586632,74.570951,1.105162,0.015308,0.088284
139,152,0.0,0.0,0.0,0,0,8149.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,9539.588019,311.0,0,0,4.724965,75.216505,1.104472,0.015194,0.072715
153,148,0.0,0.0,0.0,0,0,7921.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,9001.889442,303.0,0,0,4.760182,75.380847,1.102823,0.016229,0.091677
175,148,0.0,0.0,0.0,0,0,8172.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,8546.657900,314.0,0,0,4.675233,74.984421,1.103867,0.015840,0.064694
210,147,0.0,0.0,0.0,0,0,7853.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,8564.666282,301.0,0,0,4.619743,74.725469,1.104727,0.015428,0.072959


In [40]:
port_samples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,250,60.0,60,60,0.0,0.0,13000,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,13000.000000,500,0,0,5.972138,83.722109,1.099297,0.011968,0.057015
1,200,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,10400.000000,400,0,0,4.944851,80.892222,1.101934,0.012393,0.061607
2,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.135300,70.168720,1.104636,0.014316,0.078960
3,100,60.0,60,60,0.0,0.0,10400,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,3466.666667,400,0,0,10.887169,36.740497,5.026671,0.027286,0.263723
4,150,60.0,60,60,0.0,0.0,7800,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,7800.000000,300,0,0,3.943459,76.075343,1.102673,0.013189,0.069544
5,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.124340,70.414876,1.100336,0.014266,0.078805
6,130,60.0,60,60,0.0,0.0,6760,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,6760.000000,260,0,0,3.534727,73.555891,1.103546,0.013648,0.073640
7,115,60.0,60,60,0.0,0.0,5980,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5980.000000,230,0,0,3.329684,69.075624,1.103299,0.014540,0.077855
8,110,60.0,60,60,0.0,0.0,5720,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,5720.000000,220,0,0,3.141675,70.026340,1.100339,0.014346,0.078919
9,160,60.0,60,60,0.0,0.0,8320,26,26.0,26,0.0,0,0.0,0,0.0,2.0,0.0,8320.000000,320,0,0,4.147076,77.162799,1.101572,0.013000,0.066671


### Then fill values into columns that are not related to each other:

In [41]:
port_dataset2['Fwd Segment Size Avg'] = np.full(NUM_OF_ROWS, 2.0)
port_dataset2

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,142,0.0,0.0,0.0,0,0,7850.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,8165.647496,301.0,0,0,4.409825,73.745851,1.103588,0.015495,0.062658
1,223,0.0,0.0,0.0,0,0,11933.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,12640.427722,473.0,0,0,6.933517,85.523079,1.109021,0.022098,0.097044
2,371,0.0,0.0,0.0,0,0,20456.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,21023.384587,766.0,0,0,11.694006,88.750000,1.113000,0.037905,0.158658
3,346,0.0,0.0,0.0,0,0,18593.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,21494.831700,710.0,0,0,10.807960,88.750000,1.113000,0.036522,0.168823
4,290,0.0,0.0,0.0,0,0,15548.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,16602.231690,599.0,0,0,9.023963,88.750000,1.111691,0.030881,0.122353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,262,0.0,0.0,0.0,0,0,14411.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,16451.992033,556.0,0,0,8.123166,88.750000,1.111344,0.027300,0.146697
14996,110,0.0,0.0,0.0,0,0,5882.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,6303.303382,233.0,0,0,3.516064,69.574965,1.101330,0.011220,0.067705
14997,409,0.0,0.0,0.0,0,0,22508.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,25413.704034,870.0,0,0,12.997223,88.750000,1.113000,0.042161,0.233969
14998,263,0.0,0.0,0.0,0,0,14132.0,0.0,0.0,0.0,0,0,0,0,0,2.0,0,16469.830342,540.0,0,0,8.354511,88.750000,1.111142,0.028674,0.121273


### Then we insert data into columns that have the exact same values:

In [42]:
same_values1 = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']

# generate random values for the 'Average Packet Length' column
rand_values = np.random.randint(min_max_dict['Average Packet Length'][0]*0.95, min_max_dict['Average Packet Length'][1]*1.05, size = NUM_OF_ROWS)

# assign the random values
for col in same_values1:
    port_dataset2[col] = rand_values

In [43]:
same_values2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']

# generate random values for the 'Fwd Packet Length Max' column
rand_values = np.random.randint(min_max_dict['Fwd Packet Length Max'][0]*0.95, min_max_dict['Fwd Packet Length Max'][1]*1.05, size = NUM_OF_ROWS)

# assign the random values
for col in same_values2:
    port_dataset2[col] = rand_values

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [44]:
port_dataset2

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,142,85,85,85,0,0,7850.0,29,29,29,0,0,0,0,0,2.0,0,8165.647496,301.0,0,0,4.409825,73.745851,1.103588,0.015495,0.062658
1,223,91,91,91,0,0,11933.0,38,38,38,0,0,0,0,0,2.0,0,12640.427722,473.0,0,0,6.933517,85.523079,1.109021,0.022098,0.097044
2,371,70,70,70,0,0,20456.0,38,38,38,0,0,0,0,0,2.0,0,21023.384587,766.0,0,0,11.694006,88.750000,1.113000,0.037905,0.158658
3,346,58,58,58,0,0,18593.0,28,28,28,0,0,0,0,0,2.0,0,21494.831700,710.0,0,0,10.807960,88.750000,1.113000,0.036522,0.168823
4,290,91,91,91,0,0,15548.0,32,32,32,0,0,0,0,0,2.0,0,16602.231690,599.0,0,0,9.023963,88.750000,1.111691,0.030881,0.122353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,262,70,70,70,0,0,14411.0,30,30,30,0,0,0,0,0,2.0,0,16451.992033,556.0,0,0,8.123166,88.750000,1.111344,0.027300,0.146697
14996,110,72,72,72,0,0,5882.0,30,30,30,0,0,0,0,0,2.0,0,6303.303382,233.0,0,0,3.516064,69.574965,1.101330,0.011220,0.067705
14997,409,88,88,88,0,0,22508.0,34,34,34,0,0,0,0,0,2.0,0,25413.704034,870.0,0,0,12.997223,88.750000,1.113000,0.042161,0.233969
14998,263,89,89,89,0,0,14132.0,35,35,35,0,0,0,0,0,2.0,0,16469.830342,540.0,0,0,8.354511,88.750000,1.111142,0.028674,0.121273


In [45]:
port_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,10.000000,10.0,10.0,10.0,10.0,10.0,10.000000,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.000000,10.000000,10.0,10.0,10.000000,10.000000,10.000000,10.000000,10.000000
mean,143.500000,60.0,60.0,60.0,0.0,0.0,7982.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,7288.666667,307.000000,0.0,0.0,4.616042,70.783442,1.494430,0.014895,0.090674
std,48.422332,0.0,0.0,0.0,0.0,0.0,2535.796171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740.777667,97.530622,0.0,0.0,2.388908,12.948226,1.241104,0.004442,0.061298
min,100.000000,60.0,60.0,60.0,0.0,0.0,5720.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3466.666667,220.000000,0.0,0.0,3.124340,36.740497,1.099297,0.011968,0.057015
25%,110.000000,60.0,60.0,60.0,0.0,0.0,5785.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5720.000000,222.500000,0.0,0.0,3.188677,70.061935,1.100647,0.013047,0.067389
50%,122.500000,60.0,60.0,60.0,0.0,0.0,7280.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6370.000000,280.000000,0.0,0.0,3.739093,71.985383,1.102304,0.013957,0.075747
75%,157.500000,60.0,60.0,60.0,0.0,0.0,9880.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8190.000000,380.000000,0.0,0.0,4.745407,76.890935,1.103484,0.014338,0.078890
max,250.000000,60.0,60.0,60.0,0.0,0.0,13000.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,13000.000000,500.000000,0.0,0.0,10.887169,83.722109,5.026671,0.027286,0.263723


In [46]:
port_dataset2.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.0,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.0,15000.0,15000.0,15000.0,15000.0,15000.0,15000.0,15000.000000,15000.000000,15000.0,15000.0,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,252.040267,75.077667,75.077667,75.077667,0.0,0.0,13683.718733,31.522400,31.522400,31.522400,0.0,0.0,0.0,0.0,0.0,2.0,0.0,15174.962210,525.746933,0.0,0.0,7.997946,83.625025,1.109259,0.026640,0.131623
std,92.852305,10.732455,10.732455,10.732455,0.0,0.0,5047.721919,4.594922,4.594922,4.594922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5626.840213,193.939707,0.0,0.0,2.948309,7.056893,0.003903,0.009873,0.051865
min,90.000000,57.000000,57.000000,57.000000,0.0,0.0,4789.000000,24.000000,24.000000,24.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,5106.264754,184.000000,0.0,0.0,2.790311,66.188116,1.100000,0.008984,0.037685
25%,172.000000,66.000000,66.000000,66.000000,0.0,0.0,9356.750000,28.000000,28.000000,28.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10376.133710,359.000000,0.0,0.0,5.467030,78.679475,1.106000,0.018205,0.087979
50%,252.000000,75.000000,75.000000,75.000000,0.0,0.0,13708.500000,32.000000,32.000000,32.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,15144.555146,526.500000,0.0,0.0,8.007476,88.750000,1.110421,0.026629,0.129596
75%,333.000000,84.000000,84.000000,84.000000,0.0,0.0,18101.000000,35.000000,35.000000,35.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,20029.719214,695.000000,0.0,0.0,10.574602,88.750000,1.113000,0.035134,0.170614
max,411.000000,93.000000,93.000000,93.000000,0.0,0.0,22745.000000,39.000000,39.000000,39.000000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,26304.885770,874.000000,0.0,0.0,13.322569,88.750000,1.113000,0.046135,0.256928


---

## Adding the Label column:

In [47]:
# adding a label to the dataset
port_dataset2['Label'] = ATTACK_NAME

---

## Select the rows are want from the generated second dataset such that the 'Number of Ports' value will be >= 170:

In [48]:
port_dataset2 = port_dataset2[port_dataset2['Number of Ports'] >= 170]
print(f'Second Attack Dataset Shape Before: {port_dataset2.shape}')

Second Attack Dataset Shape Before: (11392, 27)


In [49]:
port_dataset2 = port_dataset2.sample(n=5000, random_state = 42) 
port_dataset2.shape
print(f'Second Attack Dataset Shape After: {port_dataset2.shape}')

Second Attack Dataset Shape After: (5000, 27)


---

## At the end we merge the two sample datasets together and then save it as a CSV file:

In [50]:
mergedport_dataset = pd.concat([port_dataset, port_dataset2], axis=0)
mergedport_dataset = mergedport_dataset.sample(frac=1, random_state=42).reset_index(drop=True)
print(f'Attack Dataset Shape: {mergedport_dataset.shape}')

Attack Dataset Shape: (20000, 27)


Make sure that the data in the 'Total Length of Fwd Packet' column is of type int for consistency.  

In [51]:
mergedport_dataset['Total Length of Fwd Packet'] = mergedport_dataset['Total Length of Fwd Packet'].astype(int)

In [52]:
# save the dataset
mergedport_dataset.to_csv('port_scan_closed_port_dataset.csv', index=False)